In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np

import os

In [2]:
this_dir = os.getcwd()
os.chdir('..')
model_diff = pd.read_csv( 'data/model_diff.csv' )
# bug where 'None' strategy (i.e., a string) loaded as None/NaN
model_diff['strat'] = model_diff['strat'].fillna('None')

from utils.data_processing import *

os.chdir(this_dir)

In [3]:
comp_cols = [ 'boundary_prox', 'losses', 'pvi', 'times_forgotten', 'instance_hardness', 'irt_difficulty', 'tok_len' ]


# loops through tasks (i.e., score variables / "svars") and add Spearman Correlations with CIs
for svar in model_diff['score_var'].unique():
    comp_cols = [ 'boundary_prox', 'losses', 'pvi', 'times_forgotten', 'instance_hardness', 'irt_difficulty', 'tok_len' ]

    # micro-correlation calculation
    model_cormat_wer = create_cormat(model_diff[model_diff['score_var']==svar], comp_cols)

    # micro-average is all at once
    micro_gdf_wer = calculate_micro_avg(model_cormat_wer, comp_cols)


    # format into dataframe
    micro_corr_df_wer = create_agg_corr_df(micro_gdf_wer, comp_cols)

    # add confidence intervals
    z = 1.96
    micro_corr_df_wer['ci_low'] = micro_corr_df_wer['mean_scorr'] - z*( micro_corr_df_wer['std_scorr'] / np.sqrt(micro_corr_df_wer['count_models']) )
    micro_corr_df_wer['ci_high'] = micro_corr_df_wer['mean_scorr'] + z*( micro_corr_df_wer['std_scorr'] / np.sqrt(micro_corr_df_wer['count_models']) )


    # write each dataframe to a folder for later access in R plots
    this_dir = os.getcwd()
    os.chdir('..')
    micro_corr_df_wer.to_csv( f'data/corr_by_svar/micro_corr_df-for_{svar}.csv', index=False )
    os.chdir(this_dir)

